# **NAMD 3 Complete Simulation Pipeline (GPU Accelerated)**

This notebook provides a complete pipeline for running Molecular Dynamics simulations using NAMD 3 on Google Colab with CUDA acceleration.

**Pipeline Steps:**
1.  **Setup**: Install NAMD 3 and mount Google Drive.
2.  **Configuration**: Define system files (PDB, PSF) and simulation parameters.
3.  **Minimization**: Relax the system to remove steric clashes.
4.  **Equilibration**: Heat and equilibrate the system.
5.  **Production**: Run the final production simulation.
6.  **Analysis**: Basic output handling.

## **1. Setup & Installation**

In [ ]:
# @title Check GPU Status
!nvidia-smi

In [ ]:
# @title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title Install NAMD 3 (CUDA)
# Downloads and extracts NAMD 3 alpha/beta version with CUDA support.
import os

if not os.path.exists("/content/namd3"):
    !wget "https://www.ks.uiuc.edu/Research/namd/cvs/download/741376/NAMD_Git-2025-08-02_Linux-x86_64-multicore-CUDA.tar.gz" -O namd.tar.gz
    !tar -xzf namd.tar.gz
    # Create a symlink for easier access
    !ln -s /content/NAMD_Git-2025-08-02_Linux-x86_64-multicore-CUDA/namd3 /content/namd3
    !chmod +x /content/namd3
    print("NAMD 3 installed successfully.")
else:
    print("NAMD 3 is already installed.")

## **2. System Configuration**
Define your input files and simulation parameters here. Ensure your PDB, PSF, and parameter files are uploaded to Google Drive.

In [ ]:
# @title Input Parameters

# --- File Paths (Google Drive) ---
# Base directory for your project in Drive
PROJECT_DIR = "/content/drive/MyDrive/Temp" # @param {type:"string"}

# Input files (relative to PROJECT_DIR or absolute paths)
PDB_FILE = "system.pdb" # @param {type:"string"}
PSF_FILE = "system.psf" # @param {type:"string"}

# Parameter files (list, space-separated if multiple)
# Example: "par_all36_prot.prm par_all36_lipid.prm"
PARAMETER_FILES = "par_all36_prot.prm" # @param {type:"string"}

# --- Simulation Settings ---
TEMPERATURE = 300 # @param {type:"integer"}

# Periodic Boundary Conditions (Vector dimensions)
# If you have an .xsc file, we can modify this to use it, but for now we'll define box size.
CELL_BASIS_VECTOR_1 = 60.0 # @param {type:"number"}
CELL_BASIS_VECTOR_2 = 60.0 # @param {type:"number"}
CELL_BASIS_VECTOR_3 = 60.0 # @param {type:"number"}
CELL_ORIGIN_X = 0.0 # @param {type:"number"}
CELL_ORIGIN_Y = 0.0 # @param {type:"number"}
CELL_ORIGIN_Z = 0.0 # @param {type:"number"}

import os

# Validate paths
full_pdb_path = os.path.join(PROJECT_DIR, PDB_FILE)
full_psf_path = os.path.join(PROJECT_DIR, PSF_FILE)

print(f"Project Directory: {PROJECT_DIR}")
print(f"PDB: {full_pdb_path}")
print(f"PSF: {full_psf_path}")

# Helper function to generate NAMD config header
def get_namd_header(output_name, restart_freq=500):
    params = PARAMETER_FILES.split()
    param_lines = "\n".join([f"parameters {os.path.join(PROJECT_DIR, p)}" for p in params])

    return f"""
structure          {full_psf_path}
coordinates        {full_pdb_path}

set temperature    {TEMPERATURE}
set outputname     {output_name}

paraTypeCharmm     on
{param_lines}

# Force-Field Parameters
exclude            scaled1-4
1-4scaling         1.0
cutoff             12.0
switching          on
switchdist         10.0
pairlistdist       14.0

# Integrator Parameters
timestep           2.0  ;# 2fs/step
rigidBonds         all  ;# needed for 2fs steps
nonbondedFreq      1
fullElectFrequency 2
stepspercycle      10

# Output
outputEnergies     {restart_freq}
dcdfreq            {restart_freq}
restartfreq        {restart_freq}
"""

## **3. Minimization**
Relax the system to remove high-energy steric clashes.

In [ ]:
# @title Run Minimization
MIN_STEPS = 1000 # @param {type:"integer"}

min_conf_content = get_namd_header("minimization") + f"""
# Minimization Specifics
temperature        $temperature

# Periodic Boundary Conditions
cellBasisVector1   {CELL_BASIS_VECTOR_1} 0 0
cellBasisVector2   0 {CELL_BASIS_VECTOR_2} 0
cellBasisVector3   0 0 {CELL_BASIS_VECTOR_3}
cellOrigin         {CELL_ORIGIN_X} {CELL_ORIGIN_Y} {CELL_ORIGIN_Z}
wrapAll            on

minimize           {MIN_STEPS}
"""

with open("minimization.conf", "w") as f:
    f.write(min_conf_content)

print("Running Minimization...")
!./namd3 +p 2 +devices 0 minimization.conf > minimization.log
print("Minimization Complete. Check minimization.log")
!tail -n 20 minimization.log

## **4. Equilibration**
Heat the system and equilibrate pressure/temperature.

In [ ]:
# @title Run Equilibration
EQ_STEPS = 5000 # @param {type:"integer"}

eq_conf_content = get_namd_header("equilibration") + f"""
# Input from Minimization
bincoordinates     minimization.coor
binvelocities      minimization.vel
extendedSystem     minimization.xsc

# Thermostat (Langevin)
langevin           on
langevinTemp       $temperature
langevinDamping    1
langevinHydrogen   off

# Barostat (Langevin Piston)
useGroupPressure      yes
useFlexibleCell       no
useConstantArea       no
langevinPiston        on
langevinPistonTarget  1.01325 ;#  1 atm
langevinPistonPeriod  100.0
langevinPistonDecay   50.0
langevinPistonTemp    $temperature

run {EQ_STEPS}
"""

with open("equilibration.conf", "w") as f:
    f.write(eq_conf_content)

print("Running Equilibration...")
!./namd3 +p 2 +devices 0 equilibration.conf > equilibration.log
print("Equilibration Complete. Check equilibration.log")
!tail -n 20 equilibration.log

## **5. Production Run**
Run the final simulation with optimized settings.

In [ ]:
# @title Run Production
PROD_STEPS = 50000 # @param {type:"integer"}

prod_conf_content = get_namd_header("production") + f"""
# Input from Equilibration
bincoordinates     equilibration.coor
binvelocities      equilibration.vel
extendedSystem     equilibration.xsc

# Thermostat & Barostat (Keep same as Eq)
langevin           on
langevinTemp       $temperature
langevinDamping    1
langevinHydrogen   off

useGroupPressure      yes
useFlexibleCell       no
useConstantArea       no
langevinPiston        on
langevinPistonTarget  1.01325
langevinPistonPeriod  100.0
langevinPistonDecay   50.0
langevinPistonTemp    $temperature

run {PROD_STEPS}
"""

with open("production.conf", "w") as f:
    f.write(prod_conf_content)

import multiprocessing
cores = multiprocessing.cpu_count()

print(f"Running Production with {cores} CPU cores and GPU...")
# Run with optimization flags
!./namd3 +p {cores} +setcpuaffinity +idlepoll +devices 0 production.conf > production.log
print("Production Complete.")

## **6. Save Results**
Copy all output files to Google Drive.

In [ ]:
# @title Backup to Drive
import shutil
import glob

output_files = glob.glob("*.dcd") + glob.glob("*.coor") + glob.glob("*.vel") + glob.glob("*.xsc") + glob.glob("*.log")

print(f"Copying {len(output_files)} files to {PROJECT_DIR}...")

for file in output_files:
    try:
        shutil.copy(file, PROJECT_DIR)
        print(f"Copied: {file}")
    except Exception as e:
        print(f"Error copying {file}: {e}")

print("Backup Complete.")